In [1]:
#coding:utf-8

In [2]:
# 跟「繪圖」有關的套件，如果沒有打算繪圖就不用 import
%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import math
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Input, LSTM
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.optimizers import Adam

Mounted at /content/drive


In [3]:
def plot_confusion_matrix( test_labels, results, classes, normalize=False, title="Confusion Matrix", cmap=plt.cm.Blues):
  cm = confusion_matrix(test_labels, results)
  plt.figure()
  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)
  if normalize:
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
  thresh = cm.max() / 2
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
  
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')

In [4]:
data = pd.read_csv("/content/drive/MyDrive/DLPFLP/name,outline,totalFundedUSD.csv")
data = data.drop('country_code', axis=1)
data = data.drop('funding_rounds', axis=1)
data = data.drop('status', axis=1)
data = data.drop('entity_id', axis=1)
data = data.drop('normalized_name', axis=1)
data = data.dropna()

def isFunded(x):
  if (x == 0):
    return 0
  else:
    return 1

data['getFund'] = data['funding_total_usd'].apply(lambda x: isFunded(x))
data['notGetFund'] = data['funding_total_usd'].apply(lambda x: int(not isFunded(x)))
trainfundingTotalUsd = data[['getFund', 'notGetFund']].to_numpy()
trainOverviewString = data['overview'].to_numpy()
print(trainfundingTotalUsd.shape)
print(trainOverviewString.shape)


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(235235, 2)
(235235,)


In [5]:
# 非必要，但用了會比較快
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(" Number of GPUs available: " , len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

 Number of GPUs available:  0


IndexError: ignored

In [6]:
tokenizer = Tokenizer(lower=True, num_words=25000)
tokenizer.fit_on_texts(trainOverviewString)

In [7]:
trainOverviewString = tokenizer.texts_to_sequences(trainOverviewString)
print(trainOverviewString[20909])

[3580, 2064, 690, 19, 3, 3580, 32, 1, 80, 27, 247, 4, 321, 25, 24, 135, 2, 27, 434, 6, 3580, 2, 19, 695, 4, 6858, 321, 3580, 2064, 690, 16, 288, 46, 74, 9, 46, 996, 1549, 47, 1, 67, 24, 830, 2322, 9, 44, 28, 26, 137, 818, 12, 120, 25, 1191, 7742, 4, 120, 8, 26, 103]


In [8]:
trainOverviewString = pad_sequences(trainOverviewString, padding='post', maxlen=50)
print(trainOverviewString[10])
print(trainOverviewString.shape)

[    7     5    44 16552  1244     9   184    50   432  8700  1678    14
 16552    21   196    97   184   162  1305  3506    66  4019  1305  7350
  1305     1  1305  2946     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0]
(235235, 50)


In [9]:
num_words = len(tokenizer.word_index) + 1
print(num_words)

348189


In [10]:
model = Sequential([
  Embedding(input_dim=25000, output_dim=16),
  LSTM(256),
  Dense(1024, activation='relu'),
  Dense(512, activation='relu'),
  Dense(256, activation='relu'),
  Dense(128, activation='relu'),
  Dense(64, activation='relu'),
  Dense(2, activation='softmax')
])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          400000    
                                                                 
 lstm (LSTM)                 (None, 256)               279552    
                                                                 
 dense (Dense)               (None, 1024)              263168    
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dense_3 (Dense)             (None, 128)               32896     
                                                                 
 dense_4 (Dense)             (None, 64)                8

In [12]:
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [13]:
print(trainOverviewString)
print(trainfundingTotalUsd)

[[   22   486     4 ...   503  1195   371]
 [  436   445     6 ...   551   318   211]
 [    7   410   350 ...    40    19 22134]
 ...
 [    6   904    14 ...   507     8    31]
 [  400     1   253 ...  7612  1984   339]
 [   15  5223    24 ...  1395     1  1394]]
[[1 0]
 [0 1]
 [0 1]
 ...
 [0 1]
 [0 1]
 [0 1]]


In [ ]:
print(trainOverviewString.shape)
print(trainfundingTotalUsd.shape)
train_history = model.fit(trainOverviewString, trainfundingTotalUsd, validation_split=0.2, epochs=30, batch_size=100)

(235235, 50)
(235235, 2)
Epoch 1/30
  47/1882 [..............................] - ETA: 9:26 - loss: 0.4580 - accuracy: 0.8434

In [ ]:
testOverviewString = tokenizer.texts_to_sequences(["Wetpaint technology"])
print(testOverviewString)
testOverviewString = pad_sequences(testOverviewString, padding='post', maxlen=50)
print(testOverviewString)
predict = model.predict(testOverviewString)
print(predict)